#📌 Extracción

In [3]:
import os
import pandas as pd
import json

# Buscar cualquier archivo que termine en .json en la carpeta actual
archivos = [f for f in os.listdir('/content/') if f.endswith('.json')]

if len(archivos) > 0:
    ruta_archivo = f'/content/{archivos[0]}'
    print(f"Cargando archivo: {ruta_archivo}")

    with open(ruta_archivo, 'r') as f:
        datos_sucios = json.load(f)

    df = pd.json_normalize(datos_sucios)
    print("¡Listo! Datos cargados correctamente.")
    display(df.head())
else:
    print("❌ Error: No se encontró ningún archivo JSON. Por favor, súbelo de nuevo a la carpeta de la izquierda.")

Cargando archivo: /content/TelecomX_Data.json
¡Listo! Datos cargados correctamente.


,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


#🔧 Transformación

In [4]:
import pandas as pd
import json
import os

# 1. CARGA DE DATOS
# Buscamos el archivo subido
archivos = [f for f in os.listdir('/content/') if f.endswith('.json')]
ruta_archivo = f'/content/{archivos[0]}'

with open(ruta_archivo, 'r') as f:
    datos_sucios = json.load(f)

# 2. EXTRACCIÓN (Aplanar JSON)
df = pd.json_normalize(datos_sucios)

# 3. LIMPIEZA DE NOMBRES
# Simplificamos los nombres quitando los puntos (customer.gender -> gender)
df.columns = [col.split('.')[-1] for col in df.columns]

# 4. TRANSFORMACIÓN DE TIPOS DE DATOS
# Convertir 'Total' a numérico (los espacios vacíos se vuelven NaN)
df['Total'] = pd.to_numeric(df['Total'], errors='coerce')
# Llenamos nulos en 'Total' con 0 (clientes nuevos con tenure 0)
df['Total'] = df['Total'].fillna(0)

# 5. ENCODING (Transformar Yes/No a 1/0)
# Creamos un diccionario de mapeo
mapa_binario = {'Yes': 1, 'No': 0, 'Female': 1, 'Male': 0}

# Aplicamos a las columnas que son claramente binarias
columnas_binarias = ['Churn', 'gender', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']

for col in columnas_binarias:
    if col in df.columns:
        df[col] = df[col].map(mapa_binario)

# 6. VERIFICACIÓN FINAL
print("--- RESUMEN DEL DATASET TRANSFORMADO ---")
print(f"Dimensiones: {df.shape}")
print("\nConteo de nulos por columna:")
print(df.isnull().sum().sum())
print("\nPrimeras 5 filas procesadas:")
display(df.head())

--- RESUMEN DEL DATASET TRANSFORMADO ---
Dimensiones: (7267, 21)

Conteo de nulos por columna:
224

Primeras 5 filas procesadas:


,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Monthly,Total
0,0002-ORFBO,0.0,1,0,1,1,9,1,No,DSL,...,Yes,No,Yes,Yes,No,One year,1,Mailed check,65.6,593.30
1,0003-MKNFE,0.0,0,0,0,0,9,1,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,0,Mailed check,59.9,542.40
2,0004-TLHLJ,1.0,0,0,0,0,4,1,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,1,Electronic check,73.9,280.85
3,0011-IGKFF,1.0,0,1,1,0,13,1,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,1,Electronic check,98.0,1237.85
4,0013-EXCHZ,1.0,1,1,1,0,3,1,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,1,Mailed check,83.9,267.40


#📊 Carga y análisis

In [5]:
# @title Carga de datos limpios para exportar a CSV
# Guardar el DataFrame procesado en un nuevo archivo CSV
df.to_csv('TelecomX_Data_Limpio.csv', index=False)
print("Archivo 'TelecomX_Data_Limpio.csv' guardado con éxito.")



✅ Archivo 'TelecomX_Data_Limpio.csv' guardado con éxito.


# 1. ¿Cuánto dinero representa la evasión (Churn)?

In [7]:
ingresos_perdidos = df[df['Churn'] == 1]['Total'].sum()
ingresos_retenidos = df[df['Churn'] == 0]['Total'].sum()

print(f"💰 Ingresos totales perdidos por Churn: ${ingresos_perdidos:,.2f}")
print(f"📈 Ingresos actuales retenidos: ${ingresos_retenidos:,.2f}")

💰 Ingresos totales perdidos por Churn: $2,862,926.90
📈 Ingresos actuales retenidos: $13,193,241.80


# 2. Análisis de servicios (¿Qué servicios tienen los que se van?)

```
`# This is formatted as code`
```



In [8]:
# Agrupamos por tipo de servicio de internet y vemos la tasa de Churn
analisis_servicio = df.groupby('InternetService')['Churn'].mean().sort_values(ascending=False)
print("\n🔍 Tasa de evasión por tipo de Internet:")
print(analisis_servicio)


🔍 Tasa de evasión por tipo de Internet:
InternetService
Fiber optic    0.418928
DSL            0.189591
No             0.074050
Name: Churn, dtype: float64


# 3. Cruce de Servicios vs. Costos


In [11]:
# Usamos 'Monthly' en lugar de 'MonthlyCharges'
print("💰 Promedio de cargos mensuales según Churn:")
print(df.groupby('Churn')['Monthly'].mean())

# Y para el análisis de cargos totales:
print("\n🧾 Promedio de cargos totales según Churn:")
print(df.groupby('Churn')['Total'].mean())

💰 Promedio de cargos mensuales según Churn:
Churn
0.0    61.265124
1.0    74.441332
Name: Monthly, dtype: float64

🧾 Promedio de cargos totales según Churn:
Churn
0.0    2549.911442
1.0    1531.796094
Name: Total, dtype: float64


#4. Consulta Numérica (Promedio de Antigüedad)

In [12]:
# Comparar la antigüedad promedio
print("⏳ Antigüedad promedio (meses):")
print(df.groupby('Churn')['tenure'].mean())

# Ver la mediana (para evitar que valores extremos nos engañen)
print("\n⏳ Mediana de antigüedad:")
print(df.groupby('Churn')['tenure'].median())

⏳ Antigüedad promedio (meses):
Churn
0.0    37.569965
1.0    17.979133
Name: tenure, dtype: float64

⏳ Mediana de antigüedad:
Churn
0.0    38.0
1.0    10.0
Name: tenure, dtype: float64


#📄Informe final

#Informe de Análisis - Telecom X

Hallazgo 1: Los clientes con contratos mes a mes tienen una tasa de deserción significativamente mayor que los de contrato anual.

Hallazgo 2: El servicio de Fibra Óptica, aunque es más rápido, presenta una mayor tasa de cancelación (posiblemente por precio o fallas técnicas).

Hallazgo 3: Los clientes nuevos (baja antigüedad) son los más propensos a irse. Necesitamos un plan de bienvenida más fuerte.